# Python + Pandas is love

L'idéal pour une rapport de données, la validation d'une intuition, vite fait, sur un coin de table, en mode oneshot / quick&dirty ? ~~Excel~~ [Pandas](https://pandas.pydata.org/) !

Nous allons suivre la génération d'un rapport à partir d'un _petit_ dump de données, ce qui va nous permettre de faire une visite guidée des fonctionnalités de pandas. Ce ne sera pas exhaustif, mais devrait vous permettre d'y penser la prochaine fois. 

[Version page web](https://nbviewer.jupyter.org/github/flo-dhalluin/python-pandas-talk/blob/master/pandas-talk-run.ipynb)

In [ ]:
%matplotlib inline

## La base


Sortons les outils : 

In [ ]:
import pandas as pd 
import numpy as np

### La base : Serie

Une série, est une séquence de valeurs, de type homogène (toutes du même type : String, entiers, flotants, dates ... ), __ avec un index __

In [ ]:
pd.Series([6,5,4,3,2,1], index=10 * np.arange(6)) # index par default : 0, 1 .. 


### La base : Dataframe

Un Dataframe, c'est un tableau : 
- un ensemble de Series, nommées ( les colonnes)
- qui partagent un index


In [ ]:
data = pd.DataFrame({"time": pd.date_range("2018-01-01", periods=120, freq="2H"),
                     "value": np.random.random(120),                            
                     "category": pd.Categorical(list("abcdef" * 20))}) 
data.head()

### La base : Import/Export

L'idée étant évidemment d'importer nos données ici, et pas de les générer. Pandas supporte à peu près tout les fichiers formats de données courants :
- csv 
- excel 
- hdf5 
- json/msgpack/parquet

_mais aussi_  on peut créer un dataframe directement depuis une requête SQL, ou même Big Query.

Nous allons ici utiliser la lingua franca de la data ( rappel du contexte : on est sale, on est rapide, on est pas sexys ...) : le csv. 

In [ ]:
events = pd.read_csv("events.csv", 
                     delimiter=";", 
                     index_col=0,
                     parse_dates=True) # auto magie. 

events.head(3)

Une petite vérif rapido :

In [ ]:
events.describe()

Notons bien que : il n'y a que 4 valeurs possibles pour doc_type et beaucoup de "non-valeurs" pour doc_type donc : 
on nettoye tout ça. 

In [ ]:
events.type = events.type.astype("category")
events.doc_type = events.doc_type.fillna("UNKNOWN").astype("category")

## Let's go

### Les données :

Les clients remplissent des dossiers pour soucrire à des produits, ils doivent envoyer certains justificatifs qui sont
  validés automatiquement (ou pas). events enregistre les différentes soumissions de documents. (client_uuid, correspondant à un dossier). Si la validation automatique (colonne status ) peut échouer : le client resoumet un document du même type.

### Le problème
Nous voudrions compiler un peu des stats sur le nombre de soumission, par type de document ... ( combien de soumission par dossier / type de document, le timing .. )


### Requêtes simples : par index

l'opérateur `[]` est contre-intuitif. 


sinon, pour retrouver des lignes par index :  c'est `.iloc` (index numérique ) ou `.loc` (index déclaré )

In [ ]:
events.iloc[241]  # par index de la ligne

In [ ]:
# par l'index déclaré, oh, on peut évidemment utilser la syntax slice de python 
events.loc['2017-01-10':'2017-01-11']

En fait le fonctionnement de `[]` c'est : 

- selection de colonnes : `events["type"]` 
- avec une série de boolean : filtre / requêtes 


In [ ]:
import datetime
selected = (events.doc_type == "CNI") & (events.index.date == datetime.date(2017, 1, 10))
cni_events = events[selected]

print("%d events recorded on January 10th" % len(cni_events))


Ou encore `query()` qui est parfois plus lisible, et plus puissante :

In [ ]:
# un peu plus lisible : les CNI avant le 10 janvier, en errer ( status : False)
events.query('doc_type=="CNI" & index < "20170110" & ~status').head()

## GroupBy

> Tableau croisé dynamique, c'est mieux si on en reste là.

Passons dans le vif du sujet, et moulinons un peu nos datas...

In [ ]:
# comptons les soumissions de documents par type/status. 
(events.groupby(('doc_type','status')) # groupage imbriqué : pour chaque doc_type puis le statut 
       .count())   # fonction d'aggregation 


### Split - apply/transform - combine

Insérons ici _une jolie image_ pour illuster le flux de données

In [ ]:
# utiliseons groupby pour "numéroter" les soumissions par utilisateurs


In [ ]:
events["timestamp"] = events.index # --- groupby transform pas très heureux avec les index

# groupons par dossier, puis chaque document
grouped_doc = events.groupby(('client_uuid','doc_type'))

# on transforme une colonne. ( Split - apply - combine)
events['doc_submission_count'] = grouped_doc["timestamp"].transform(np.argsort)
events['time_since_first'] = events.groupby('client_uuid')["timestamp"].transform(lambda ts: ts-ts[0])

events.head()

In [ ]:
# Nb moyen de soumissions de doc avant succés : PAR Type de doc.
(events[events.status==True] # seulement les soumissions ok
     .groupby('doc_type')    # toujours le petit groupby ... 
     .doc_submission_count   # quantième soumission
     .aggregate(['mean', 'max', 'min'])) # aggregation des stats.. 

### Des graphiques ! 

In [ ]:
# temps passé par dossier

(events.groupby('client_uuid') # Tu vois ce que je veux dire ? 
     .time_since_first # juste le temps depuis la première action
     .max().map(lambda x: x.total_seconds()) # timedelta -> "float" ( seconds )
     .hist(bins=50)) # KABOOM !


### Encore plus de trucs fun: join et pivot. 

... To be continued

- [sources](https://github.com/flo-dhalluin/python-pandas-talk)
- fait avec [jupyter / notebook](https://jupyter.org/)
- Le saviez vous ? vous pouvez exporter un notebook en slides/reveal.js
